In [ ]:
import sqlite3

In [ ]:
dbpath="/home/sergei/.qif-transaction-generator/db.sqlite"
conn = sqlite3.connect(dbpath)
c = conn.cursor()

In [ ]:
c.execute("""
SELECT
	r.raw, i.*, a.*
FROM
	receipts r 
	join items i 
	on r.id = i.receipt_id
	join accounts a 
	on i.account_guid = a.guid 
WHERE
	purchase_date > '2022-01-01'
	and raw like '%\"user\":\"ООО \\\"Лента\\\"%'""")
data = c.fetchall()

In [ ]:
import pandas as pd